In [ ]:
import os
from glob import glob
import cv2
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from unet.model.architecture import create_unet_model
from unet.model.constants import *

# Get image paths and inspect some images

Note that the images are the training input features and that the masks are the targets.

In [ ]:
train_image_files = glob('../data/images/*')
train_mask_files = glob('../data/masks/*')
print("Number of train files: {}".format(len(train_image_files)))
print("Number of train masks: {}".format(len(train_mask_files)))

In [ ]:
sample_image_path = train_image_files[-1]
sample_image = cv2.imread(sample_image_path)
sample_mask_path = train_mask_files[-1]
sample_mask = cv2.imread(sample_mask_path)

In [ ]:
sample_image.shape, sample_mask.shape

In [ ]:
nr_samples = 5
fig, axes = plt.subplots(nr_samples, 2, figsize=(10, 5*nr_samples))

for (ax_image, ax_mask), image_path, mask_path in zip(axes, train_image_files, train_mask_files):
    ax_image.imshow(cv2.imread(image_path)[:,:,0], cmap='seismic', interpolation='bilinear')
    ax_mask.imshow(cv2.imread(mask_path)[:,:,0], cmap='gray', interpolation='bilinear')

# Inspect data

### Training Features

In [ ]:
Xtrain = [cv2.imread(p) for p in tqdm(train_image_files)]

In [ ]:
hist = cv2.calcHist([Xtrain[0]], [0], None, [256], [0,256])

In [ ]:
# note that opencv uses BGR order of channels instead of RGB

fig, ax = plt.subplots(1, 1, figsize=(20, 5))

color = ('b', 'g', 'r')
for i, col in enumerate(color):
    hist = cv2.calcHist(images=[Xtrain[0]], channels=[i], mask=None, histSize=[256], ranges=[0, 256])
    ax.plot(hist, color=col, lw=2, alpha=0.5)
ax.set_ylabel("count", fontsize=14)
ax.set_xlabel("pixel value", fontsize=14)
ax.set_xlim([0,256]);

In [ ]:
all_hists = [cv2.calcHist(images=[img[0]], channels=[0], mask=None, histSize=[256], ranges=[0, 256]) for img in Xtrain]
all_hists = np.squeeze(np.array(all_hists))


fig, ax = plt.subplots(1, 1, figsize=(20, 7))
ax.set_title("Distribution pixel values training data", fontsize=16)


ax.plot(all_hists.mean(axis=0), color='blue', lw=2)
ax.fill_between(range(256), all_hists.mean(axis=0) - all_hists.std(axis=0), all_hists.mean(axis=0) + all_hists.std(axis=0), alpha=0.7)
ax.set_ylabel("count", fontsize=14)
ax.set_xlabel("pixel value", fontsize=14)
ax.set_xlim([-1, 256]);

### Target Values

In [ ]:
ytrain = [cv2.imread(p) for p in tqdm(train_mask_files)]

In [ ]:
salt_proportions = np.array([(img.sum() / 255) / img.size for img in ytrain])

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(20, 7))
fig.tight_layout(h_pad=3)

ax1.set_title("Distribution proportion salt (all targets)", fontsize=16)
ax1.hist(salt_proportions, bins=50)

ax2.set_title("Distribution proportion salt (>1% salt coverage)", fontsize=16)
ax2.hist(salt_proportions[salt_proportions > 0.01], bins=50);

# Create the model

In [ ]:
input_image = keras.layers.Input((sample_image.shape[0], sample_image.shape[1], 1), name='image')
unet_model = create_unet_model(input_image, batchnorm=True)
unet_model.summary()

# Predict with untrained model

Just as a demonstration to see what will happen and what the outputs will be, we'll predict with the untrained model and plot the result.

In [ ]:
example_image = Xtrain[4][:,:,:1]
example_target = ytrain[4][:,:,:1]

prediction = unet_model.predict(np.array([example_image]) / 255)[0]
prediction_mask = np.zeros(shape=prediction[:,:,0].shape)
prediction_mask[prediction[:,:,0] > 0.5] = 255

In [ ]:
fig, [ax1, ax2, ax3, ax4] = plt.subplots(1, 4, figsize=(25, 10))


ax1.set_title("Input image", fontsize=14)
ax1.imshow(example_image[:,:,0], cmap='seismic', interpolation='bilinear')
ax2.set_title("True target/mask", fontsize=14)
ax2.imshow(example_target[:,:,0], cmap='gray', interpolation='bilinear')
ax3.set_title("Raw prediction", fontsize=14)
ax3.imshow(prediction[:,:,0], cmap='gray', interpolation='bilinear')
ax4.set_title("Prediction mask (threshold 0.5)", fontsize=14)
ax4.imshow(prediction_mask, cmap='gray', interpolation='bilinear');

Clearly the model still needs to learn, but at least we have seen the model perform predictions and know the architecture can handle the inputs.